# ASSIGNMENT: Cartoon Face Mask
## This assignment tests your ability to apply face recognition, image masks, image thresholding, video capturing and saving video feed into a video file concepts.

## TASK: Cartoonify faces in video feed from live webcam

### Steps
- 1. **Capture video** feed from webcam
- 2. **Recognize faces** in the video
- 3. **Replace/Mask the face** region with your favorite cartoon character
- 4. **Save the video** feed into a video file
- 5. Submit the following files
    - Notebook (.ipynb)
    - Video (.avi/.mp4)

In [5]:
import cv2
import os

# Step 1: Capture video feed from webcam
video_capture = cv2.VideoCapture(0)

# Step 2: Load face and eye cascade classifiers
cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
cascPatheyes = os.path.dirname(cv2.__file__) + "/data/haarcascade_eye_tree_eyeglasses.xml"
faceCascade = cv2.CascadeClassifier(cascPathface)
eyeCascade = cv2.CascadeClassifier(cascPatheyes)

# Step 3: Load cartoon character mask
mask_image = cv2.imread('C:/Users/Admin/Downloads/popeye.png')  # Update with your favorite cartoon character

# Step 4: Define video writer object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('cartoonified_video.mp4', fourcc, 20.0, (640, 480))  # Update resolution as needed

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60))

    for (x, y, w, h) in faces:
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Extract face ROI
        faceROI = frame[y:y+h, x:x+w]

        # Detect eyes
        eyes = eyeCascade.detectMultiScale(faceROI)

        # Draw circles around eyes
        for (ex, ey, ew, eh) in eyes:
            eye_center = (x + ex + ew // 2, y + ey + eh // 2)
            radius = int(round((ew + eh) * 0.25))
            cv2.circle(frame, eye_center, radius, (255, 0, 0), 4)

        # Apply cartoon character mask
        if mask_image is not None:
            mask_image_resized = cv2.resize(mask_image, (w, h))
            mask_image_gray = cv2.cvtColor(mask_image_resized, cv2.COLOR_BGR2GRAY)
            _, face_mask = cv2.threshold(mask_image_gray, 25, 255, cv2.THRESH_BINARY_INV)
            face_mask_no_face = cv2.bitwise_and(faceROI, faceROI, mask=face_mask)
            cartoonified_face = cv2.add(face_mask_no_face, mask_image_resized)
            frame[y:y+h, x:x+w] = cartoonified_face

    # Display the resulting frame
    cv2.imshow('Cartoonified Video', frame)
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and writer objects
video_capture.release()
out.release()
cv2.destroyAllWindows()
